Нейро-консультант по нормативам и ценам из PDF документов

In [ ]:
# !pip install -q cohere tiktoken openai langchain fastapi kaleido uvicorn pikepdf \
#              unstructured unstructured-inference pdf2image pdfminer.six faiss-cpu \
#              pydantic pydantic-settings
# !pip install pillow-heif
# !pip install pypdf

In [ ]:
from langchain.chains.router import MultiRetrievalQAChain
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import requests
import os

# --------------------------------------------------------------
from pydantic import SecretStr
from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_file='input',
                                      env_file_encoding='utf-8')
    openai_api_key: SecretStr

os.environ['OPENAI_API_KEY'] = Settings().openai_api_key.get_secret_value()

In [ ]:
# функция принимает URL PDF-файла и путь, по которому файл будет сохранен в файловой системе Colab.
# Она скачивает файл и сохраняет его, если HTTP-запрос успешен
def download_pdf_from_url(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"PDF downloaded and saved as {save_path}")
    else:
        print(f"Failed to download PDF. HTTP Status Code: {response.status_code}")


url = "https://files.stroyinf.ru/Data2/1/4295550/4293590590.pdf" # Изменена
save_path = "content/norm1.pdf"
download_pdf_from_url(url, save_path)
url = "https://files.stroyinf.ru/Data2/1/4225946/4294936957.pdf" # Изменена
save_path = "content/norm2.pdf"
download_pdf_from_url(url, save_path)
url = "https://xn---33-5cvbekc8dd5g.xn--p1ai/ckfinder/userfiles/files/price%20ot%202020-03-01.pdf" # Изменена
save_path = "content/price.pdf"
download_pdf_from_url(url, save_path)

In [ ]:
# функция, которая принимает список названий PDF-файлов и возвращает базу знаний FAISS
def create_faiss_vectorstore(pdf_names, pdf_folder_path='content/'):
    # Фильтрация файлов: загрузка только файлов с определенными именами и расширением .pdf
    all_files = os.listdir(pdf_folder_path)
    pdf_files = [fn for fn in all_files if fn in pdf_names and fn.endswith('.pdf')]

    # Создание объектов UnstructuredPDFLoader для каждого PDF-файла
    loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in pdf_files]

    all_documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                                   chunk_overlap=100,
                                                   length_function=len)
    for loader in loaders:
        # Загрузка и разделение текста
        raw_documents = loader.load()
        documents = text_splitter.split_documents(raw_documents)
        all_documents.extend(documents)

    # Создание FAISS vectorstore
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(all_documents, embeddings).as_retriever()
    return vectorstore


# Создаем БЗ по каждой спецификации (pdf)
db_norm = create_faiss_vectorstore(['norm1.pdf', 'norm2.pdf'])
db_price = create_faiss_vectorstore(['price.pdf'])

In [ ]:
db_norm.get_relevant_documents("Расскажи о показателях безопасности \
питьевой воды по микробиологическим и паразитологическим показателям")

[Document(page_content='и нормативы качества питьевой воды\n\n3.1.\n\nПитьевая вода должна быть безопасна в эпидемическом и\n\nрадиационном отношении, безвредна по химическому составу и иметь благоприятные органолептические свойства.\n\n9\n\nСанПиН 2.1.4.1074—01\n\n3.2. Качество питьевой воды должно соответствовать гигиени\xad ческим нормативам перед ее поступлением в распределительную сеть, а также в точках водоразбора наружной и внутренней водо\xad проводной сети.\n\n3.3. Безопасность питьевой воды в эпидемическом отношении определяется ее соответствием нормативам по микробиологическим и паразитологическим показателям, представленным в табл. 1.\n\nТаблица 1\n\nПоказатели Термотолерантные колиформ- ные бактерии Общие колиформные бактерии 2> Число бактерий в 100 мл о Отсутствие Общее микробное число 2> Не более 50\n\nЕдиницы измерения', metadata={'source': 'content/norm2.pdf'}),
 Document(page_content='процессе обработки воды, в таблице 5;\n\nорганолептических свойств питьевой воды в т

In [ ]:
db_price.get_relevant_documents('Какая цена (руб) у воды: "Моя вода" негаз 1,5л за штуку?')

[Document(page_content='Вода "Аква-Вита" газ\n\n1,5л\n\n6\n\n12,50\n\n75,00\n\n9\n\nВода "Аква-Вита" негаз\n\n1,5л\n\n6\n\n12,50\n\n75,00\n\n10\n\nВода "Моя вода" газ\n\n1,5л\n\n6\n\n12,00\n\n72,00\n\n11\n\nВода "Моя вода" негаз\n\n1,5л\n\n6\n\n12,00\n\n72,00\n\n12\n\nВода дистилированная\n\n1,5л\n\n6\n\n22,00\n\n132,00\n\n13\n\nВода "Аква-Вита" негаз\n\n5л\n\n2\n\n32,00\n\n64,00\n\n14\n\nВода "Дистилированная"\n\n5л\n\n2\n\n48,00\n\n96,00\n\n15\n\nВода "Моя вода" негаз\n\n5л\n\n2\n\n28,00\n\n56,00\n\n16\n\nВода минеральная природная питьевая столовая « Аква Вита» (торговый знак "Моя Вода" 1 состав (синие крышки) Тара ПЭТ\n\n18,9\n\n1\n\n85,00\n\n17\n\nВода минеральная природная питьевая столовая « Аква Вита» (торговый знак "Моя Вода" 2 состав (белые крышки) Тара ПЭТ\n\n18,9\n\n1\n\n95,00\n\n18\n\nВода минеральная природная питьевая столовая « Аква Вита» (торговый знак "Моя Вода" 3 состав(желтые крышки) Тара ПЭТ\n\n18,9\n\n1', metadata={'source': 'content/price.pdf'}),
 Document(page_c

In [ ]:
# Добавляем информацию и описание для наших векторных хранилищ (баз знаний)
retriever_info = [
    {
        "name": "Нормативы качества питьевой воды",
        "description": "Применяется для ответов на вопросы по нормативам качества и безопасности питьевой воды",
        "retriever": db_norm
    },
    {
        "name": "Прайс лист (цены) на воду и др. номенклатуру",
        "description": 'Применяется для ответов на вопросы по ценам на воду и др. номенклатуру ООО Аква-Вита',
        "retriever": db_price
    }
]

chain = MultiRetrievalQAChain.from_retrievers(ChatOpenAI(temperature=0),
                                              retriever_info,
                                              verbose=True)

In [ ]:
chain.invoke("Расскажи о показателях безопасности питьевой \
воды по микробиологическим и паразитологическим показателям")['result']



> Entering new MultiRetrievalQAChain chain...


C:\Users\alexk\AppData\Roaming\Python\Python311\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Нормативы качества питьевой воды: {'query': 'Расскажи о показателях безопасности питьевой воды по микробиологическим и паразитологическим показателям'}
> Finished chain.


'По микробиологическим и паразитологическим показателям безопасности питьевой воды, она должна соответствовать определенным нормативам. В таблице 1 приведены показатели, которые определяют безопасность питьевой воды в эпидемическом отношении. Эти показатели включают термотолерантные колиформные бактерии, общие колиформные бактерии, число бактерий в 100 мл, общее микробное число. Все эти показатели должны быть в пределах определенных норм.\n\nОпределение микробиологических и паразитологических показателей проводится с использованием методов, указанных в таблице 1. Для централизованных систем питьевого водоснабжения используются методы, указанные в [2]* и [3]*. Для нецентрализованных систем питьевого водоснабжения применяются методы, установленные в ГОСТ 18963.\n\nИсследования воды на наличие патогенных микроорганизмов должны проводиться только в лабораториях, имеющих соответствующие санитарно-эпидемиологическое заключение и лицензию на деятельность, связанную с использованием возбудител

In [ ]:
chain.invoke("Безопасность питьевой воды по микробиологическим и паразитологическим показателям. \
Нужны данные, в том числе Единицы измерения и Нормативы по показателям: \
Бактерии, Общее микробное число, Колифаги, Споры сульфитредуцирующих клостридий, Цисты лямблий")['result']



> Entering new MultiRetrievalQAChain chain...


C:\Users\alexk\AppData\Roaming\Python\Python311\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Нормативы качества питьевой воды: {'query': 'Безопасность питьевой воды по микробиологическим и паразитологическим показателям. Нужны данные, в том числе Единицы измерения и Нормативы по показателям: Бактерии, Общее микробное число, Колифаги, Споры сульфитредуцирующих клостридий, Цисты лямблий'}
> Finished chain.


'По микробиологическим и паразитологическим показателям безопасности питьевой воды, следующие данные и нормативы применяются:\n\n1. Бактерии:\n- Термотолерантные колиформные бактерии: Отсутствие\n- Общие колиформные бактерии: Не более 50 бактерий в 100 мл\n\n2. Общее микробное число: Не более 50 бляшкообразующих единиц (БОЕ) в 100 мл\n\n3. Колифаги: Отсутствие\n\n4. Споры сульфитредуцирующих клостридий: Отсутствие\n\n5. Цисты лямблий: Отсутствие\n\nЕдиницы измерения:\n- Бактерии: Число бактерий в 100 мл\n- Общее микробное число: Число образующих колонии бактерий в 1 мл\n- Колифаги: Число бляшкообразующих единиц (БОЕ) в 100 мл\n- Споры сульфитредуцирующих клостридий: Число спор в 20 мл\n- Цисты лямблий: Число цист в 50 л\n\nПримечания:\n- При определении проводится трехкратное исследование по 100 мл отобранной пробы воды.\n- Превышение норматива не допускается в 95% проб, отбираемых в точках водоразбора наружной и внутренней водопроводной сети в течение 12 месяцев, при количестве исслед

In [ ]:
chain.invoke('Какая цена (руб) у воды: "Моя вода" негаз 1,5л за штуку?')['result']



> Entering new MultiRetrievalQAChain chain...


C:\Users\alexk\AppData\Roaming\Python\Python311\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Прайс лист (цены) на воду и др. номенклатуру: {'query': "Какова цена (в рублях) на воду 'Моя вода' негазированную 1,5л за штуку?"}
> Finished chain.


'Цена на воду "Моя вода" негазированную 1,5л за штуку составляет 12,50 рублей.'

In [ ]:
chain.invoke('Какие цены у воды в зависимости от емкости: "Моя вода" негазированная за штуку?')['result']



> Entering new MultiRetrievalQAChain chain...


C:\Users\alexk\AppData\Roaming\Python\Python311\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Прайс лист (цены) на воду и др. номенклатуру: {'query': 'Какие цены у воды в зависимости от емкости: "Моя вода" негазированная за штуку?'}
> Finished chain.


'Цены на воду "Моя вода" негазированную в зависимости от емкости:\n\n- 0,5л - 11,50 руб за штуку\n- 1,5л - 12,50 руб за штуку\n- 5л - 28,00 руб за штуку\n\nПожалуйста, обратите внимание, что это цены указаны за одну штуку продукта.'